In [1]:
import altair as alt
import pandas as pd
from vega_datasets import data
import time
import ipywidgets as widgets
from ipywidgets import interact
#from IPython.display import display
#alt.themes.enable('opaque')

dataset = pd.read_csv('./LPD2022_public.csv', encoding='latin-1', dtype={'M_biome': str})

countries = alt.topo_feature(data.world_110m.url, 'countries')

alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [12]:
background = alt.Chart(countries).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project(
    "equirectangular"
).properties(
    width=500,
    height=300
)

slider = alt.binding_range(min=1950, max=2020, step=1, name='Year:')
year = alt.param(value=1950, bind=slider)

options = ['Actinopteri', 'Amphibia', 'Aves', 'Dipneusti', 'Elasmobranchii', 'Holocephali', 'Mammalia', 'Myxini', 'Petromyzonti', 'Repitalia']
labels = [option + ' ' for option in options]

input_dropdown = alt.binding_radio(
    options=options + [None],
    labels=labels + ['All'],
    name='Class: '
)

selection = alt.selection_point(
    fields=['Class'],
    bind=input_dropdown,
)

def demo(i):
    variable = str(i) + ":Q"

    map = alt.Chart(dataset).mark_circle().encode(
        longitude='Longitude:Q',
        latitude='Latitude:Q',
        color = alt.Color('Class:N').scale(domain=options),
        size=variable,
        tooltip=['Common_name', variable]
    ).add_params(
        selection
    ).transform_filter(
        selection
    )

    bars = alt.Chart(dataset).mark_bar().encode(
        x=variable,
        y=alt.Y('Class:N').sort('-x'),
        color = alt.Color('Class:N').scale(domain=options),
        tooltip=['Common_name', variable]
    ).add_params(
        selection
    ).transform_filter(
        selection
    )
    return alt.hconcat(background + map, bars)

interact(demo, i = widgets.IntSlider(
    value=1950,
    min=1950,
    max=2020,
    step=1,
    description='Year:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
))

interactive(children=(IntSlider(value=1950, continuous_update=False, description='Year:', max=2020, min=1950),…

<function __main__.demo(i)>